In [17]:
# 1. Importar librerías
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from IPython.display import display, clear_output
from ipywidgets import widgets

# Configurar renderizador de Plotly
pio.renderers.default = "browser"

# 2. Cargar los archivos CSV
try:
    alumnos = pd.read_csv('csv/alumnos.csv')
    carreras = pd.read_csv('csv/carreras.csv')
    predicciones = pd.read_csv('csv/predicciones.csv')
except FileNotFoundError as e:
    print(f"Error: No se encontró el archivo {e}. Asegúrate de que los archivos estén en la carpeta 'csv'.")
    raise

# 3. Renombrar columnas
alumnos.rename(columns={'nombre': 'nombre_alumno'}, inplace=True)
carreras.rename(columns={'nombre': 'nombre_carrera'}, inplace=True)

# 4. Convertir id_alumno a tipo numérico (Int64) para evitar errores
alumnos['id_alumno'] = pd.to_numeric(alumnos['id_alumno'], errors='coerce').astype('Int64')
predicciones['id_alumno'] = pd.to_numeric(predicciones['id_alumno'], errors='coerce').astype('Int64')

# 5. Unir alumnos con carreras
df_merged = alumnos.merge(
    carreras[['_id', 'nombre_carrera']],
    left_on='id_carrera',
    right_on='_id',
    how='left'
)

# 6. Unir con predicciones
df_combined = df_merged.merge(
    predicciones[['id_alumno', 'riesgo']],
    on='id_alumno',
    how='left'
)

# 7. Limpiar datos
df_combined['riesgo'] = df_combined['riesgo'].fillna('Sin riesgo')
df_combined['nombre_carrera'] = df_combined['nombre_carrera'].fillna('Sin carrera')

# 8. Gráfico interactivo con Plotly
def graficar_interactivo(df):
    # Preparar datos
    orden = ['Bajo', 'Sin riesgo']
    conteo = df['riesgo'].value_counts().reindex(orden, fill_value=0)
    df_conteo = conteo.reset_index()
    df_conteo.columns = ['riesgo', 'Cantidad']

    # Crear gráfico
    fig = px.bar(
        df_conteo,
        x='riesgo',
        y='Cantidad',
        text='Cantidad',
        color='riesgo',
        title='Distribución de riesgo',
        color_discrete_sequence=['#77dd77', '#ffb347', '#ff6961', '#92c8c8']
    )

    fig.update_traces(textposition='outside')
    fig.update_layout(
        xaxis_title='Nivel de riesgo',
        yaxis_title='Cantidad de estudiantes',
        uniformtext_minsize=10,
        uniformtext_mode='hide'
    )

    return fig

# 9. Dropdown para seleccionar carrera
carreras_lista = ['Todas'] + df_combined['nombre_carrera'].dropna().unique().tolist()
selector_carrera = widgets.Dropdown(
    options=carreras_lista,
    value='Todas',
    description='Carrera:'
)

# 10. Output para mostrar gráfico y tabla
output = widgets.Output()

# 11. Función interactiva
def actualizar_carrera(carrera):
    with output:
        output.clear_output()

        # Filtrar por carrera
        if carrera == 'Todas':
            datos = df_combined
        else:
            datos = df_combined[df_combined['nombre_carrera'] == carrera]

        # Mostrar gráfico
        fig = graficar_interactivo(datos)
        fig.show()

        # Mostrar tabla al hacer clic en el nivel de riesgo
        def mostrar_alumnos(riesgo):
            clear_output(wait=True)
            print(f"\nAlumnos con riesgo '{riesgo}':")
            display(datos[datos['riesgo'] == riesgo][['id_alumno', 'nombre_alumno', 'nombre_carrera', 'riesgo']])

        # Dropdown para seleccionar riesgo
        selector_riesgo = widgets.Dropdown(
            options=sorted(datos['riesgo'].unique()),
            description='Riesgo:',
            value='Bajo'  # Valor predeterminado
        )

        # Conectar el dropdown al evento de cambio
        selector_riesgo.observe(lambda change: mostrar_alumnos(change.new), names='value')

        # Mostrar el dropdown de riesgo
        display(selector_riesgo)

# 12. Conectar el dropdown a la función
widgets.interact(actualizar_carrera, carrera=selector_carrera)
display(output)

interactive(children=(Dropdown(description='Carrera:', options=('Todas', 'Ingeniería en Redes Inteligentes y C…

Output()